In [23]:
# libs
import pandas as pd
from datetime import datetime

# stats R type shit 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from statsmodels.api import OLS
import statsmodels.api as sm

In [24]:
# today's date string for file namine automation
today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
print(today_date_str)

_2024-02-23.csv


In [25]:
# cocncat base file name with today's date string
filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"

# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. stocks trading w/ > 0% pre-market price gap change')
trading_view_raw_df.head(5)

6553 U.S. stocks trading w/ > 0% pre-market price gap change


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,...,Relative Volume 1 week,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Beta 1 year,Beta 3 years,Beta 5 years
0,MSFT,Microsoft Corporation,NASDAQ,3049005077458.27,USD,410.34,USD,4.04,USD,0.98141625,...,0.73104368,1.684720,1.407963,1.572144,411.723333,37.11233912,Technology Services,0.98372686,1.0329885,0.8991026
1,AAPL,Apple Inc.,NASDAQ,2818452019809.7324,USD,182.52,USD,0.63,USD,0.34170418,...,0.72922452,1.542007,1.373282,1.651346,183.263333,28.39894196,Electronic Technology,1.0909481,1.2225922,1.3124571
2,NVDA,NVIDIA Corporation,NASDAQ,1946779912103.8457,USD,788.17,USD,22.43,USD,2.85594235,...,1.46836546,6.218899,5.576206,3.700871,795.936667,66.03024337,Electronic Technology,1.5339884,2.2734668,1.6840769
3,AMZN,"Amazon.com, Inc.",NASDAQ,1817687795647.0127,USD,174.99,USD,-0.32,USD,-0.18329706,...,0.87591062,1.180196,1.646224,2.062945,174.813333,60.55855482,Retail Trade,1.144756,1.3340302,1.142365
4,GOOG,Alphabet Inc.,NASDAQ,1798075043791.659,USD,145.29,USD,-0.4,USD,-0.27525461,...,0.69464835,0.804614,1.394955,1.784666,145.345000,25.0275615,Technology Services,1.1141071,1.2103225,1.0635216


END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END

In [12]:
# first 10 rows df
trading_view_raw_df_testing = trading_view_raw_df.head(10)
# save to csv
trading_view_raw_df_testing.to_csv('trading_view_raw_data/tv_screen_gap-up_testing.csv', index=False)
# make sure data is there at least
print(len(trading_view_raw_df_testing.index))

10


In [17]:
# baseline raw cols from trading view initial filter
features_raw_all = trading_view_raw_df.columns
# convert to list
features_raw_all = features_raw_all.tolist()
# look at the trading view columns
print(features_raw_all)

['Symbol', 'Description', 'Exchange', 'Market capitalization', 'Market capitalization - Currency', 'Price', 'Price - Currency', 'Post-market Change', 'Post-market Change - Currency', 'Post-market Change %', 'Post-market Close', 'Post-market Close - Currency', 'Float shares outstanding', 'Volume 1 day', 'Relative Volume 1 day', 'Average Volume 10 days', 'Post-market Volume', 'Relative Volume at Time', 'Relative Volume at Time.1', 'Volume 1 week', 'Relative Volume 1 week', 'Volatility 1 day', 'Volatility 1 week', 'Volatility 1 month', 'Volume Weighted Average Price 1 day', 'Price to earnings ratio', 'Sector', 'Beta 1 year', 'Beta 3 years', 'Beta 5 years']


In [20]:
# feature selection
# features = ['Market capitalization', 'Volume 1 day', 'Volatility 1 week', 'Beta 1 year', 'Sector']
features = features_raw_all
target = 'Post-market Change %'

# Preprocessing data: Drop rows with missing values in the selected columns
tv_clean_df = trading_view_raw_df[features + [target]].dropna()

# Check the unique values and their frequency for the target variable
print(tv_clean_df['Post-market Change %'].value_counts())


# # Encoding categorical variables and scaling numerical variables
# categorical_features = ['Sector']
# numeric_features = list(set(features) - set(categorical_features))

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), numeric_features),
#         ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
#     ])

# # Prepare the data for modeling
# X = tv_clean_df[features]
# y = tv_clean_df[target]

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define and fit the model
# model = Pipeline(steps=[('preprocessor', preprocessor),
#                         ('regressor', LinearRegression())])

# model.fit(X_train, y_train)

# # Predict and evaluate the model
# y_pred = model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'MSE: {mse}')
# print(f'R^2: {r2}')

# # For detailed statistics, we need to fit the model using statsmodels
# # We'll prepare the data again because statsmodels requires it to be done differently
# X_train_transformed = model.named_steps['preprocessor'].transform(X_train)
# X_train_transformed = sm.add_constant(X_train_transformed)  # Adding a constant for the intercept

# # Fit the model using OLS
# ols_model = sm.OLS(y_train, X_train_transformed).fit()

# # Print the summary of the model to see detailed statistics
# print(ols_model.summary())


ValueError: Grouper for 'Post-market Change %' not 1-dimensional